In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from google.cloud import bigquery
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from google.cloud import storage
from tensorflow.keras import callbacks
from tensorflow.keras import Sequential, Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

2025-06-04 17:13:51.614073: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 17:13:51.634491: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 17:13:51.785257: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 17:13:51.902095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749053632.005157  101038 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749053632.03

In [3]:
# Cloud Storage
BUCKET_NAME="skin_scan_mohnatz"

In [4]:
def get_data_from_bq(
        gcp_project:str,
        query:str,
        data_has_header=True
    ) -> pd.DataFrame:
    """
    Retrieve `query` data from BigQuery
    """
    client = bigquery.Client(project=gcp_project)
    query_job = client.query(query)
    result = query_job.result()
    df = result.to_dataframe()
    return df

GCP_PROJECT = "skin-scan-461716"
BQ_DATASET = "skin_scan"

query_28 = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.train_28_28
    """

In [5]:
data = get_data_from_bq(f"{GCP_PROJECT}", query_28)
data = data.astype("float64")

In [6]:
query_metadata = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.train_metadata
    """

In [7]:
metadata = get_data_from_bq(f"{GCP_PROJECT}", query_metadata)

In [8]:
data.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label,index
0,3.0,1.0,6.0,3.0,1.0,6.0,2.0,1.0,6.0,4.0,...,63.0,82.0,61.0,56.0,76.0,56.0,52.0,70.0,2.0,529.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,60.0,69.0,95.0,79.0,99.0,79.0,62.0,73.0,2.0,921.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,1.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,6.0,1813.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,1.0,4.0,3.0,1.0,3.0,0.0,0.0,0.0,4.0,7811.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,...,2.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,4.0,7214.0


In [9]:
data.sort_values(by="index",inplace=True)
data.drop(columns=["index","label"],inplace=True)
data.reset_index(drop=True,inplace=True)
data = data/255

In [10]:
data.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2342,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351
0,0.752941,0.600000,0.756863,0.764706,0.607843,0.752941,0.772549,0.603922,0.725490,0.792157,...,0.525490,0.678431,0.486275,0.541176,0.717647,0.576471,0.650980,0.725490,0.603922,0.694118
1,0.098039,0.054902,0.117647,0.266667,0.188235,0.294118,0.482353,0.364706,0.494118,0.619608,...,0.321569,0.235294,0.152941,0.215686,0.098039,0.054902,0.109804,0.098039,0.054902,0.105882
2,0.752941,0.541176,0.600000,0.784314,0.568627,0.639216,0.788235,0.556863,0.627451,0.807843,...,0.584314,0.654902,0.505882,0.560784,0.623529,0.486275,0.556863,0.533333,0.407843,0.458824
3,0.149020,0.074510,0.117647,0.372549,0.231373,0.282353,0.560784,0.403922,0.466667,0.670588,...,0.286275,0.172549,0.101961,0.141176,0.098039,0.047059,0.066667,0.098039,0.047059,0.058824
4,0.619608,0.443137,0.545098,0.760784,0.564706,0.682353,0.843137,0.635294,0.749020,0.882353,...,0.788235,0.819608,0.650980,0.725490,0.674510,0.529412,0.584314,0.427451,0.305882,0.360784


In [11]:
metadata.sort_values(by="index",inplace=True)
metadata.drop(columns="index",inplace=True)
metadata.reset_index(drop=True,inplace=True)

In [12]:
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
test_array = np.array(data.iloc[4,:])

In [ ]:
test_array = test_array.reshape((28,28,3))
test_array.shape
plt.imshow(test_array)

In [ ]:
def print_28_28(index):
    temp_array = np.array(data.iloc[index,:]).reshape((28,28,3))
    image_id = metadata.iloc[index,:].image_id
    plt.imshow(temp_array)
    return image_id

In [ ]:
print_28_28(150)

In [ ]:
from google.cloud import storage
from PIL import Image
import io
import matplotlib.pyplot as plt

# Set up the client
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

blobs = bucket.list_blobs(prefix="train_all_images/")
images = [blob.name for blob in blobs if blob.name.lower().endswith(".jpg")]

In [ ]:
len(images)

In [ ]:
pixel_list = []

In [ ]:
test_image = images[0]

In [ ]:
metadata[["dx"]].value_counts()

In [ ]:
dx_dict = {"nv": 4,
           "mel":6,
           "bkl":2,
           "bcc":1,
           "akiec":0,
           "vasc":5,
           "df":3   
}

In [ ]:
metadata["target"] = metadata.dx.map(dx_dict)

In [ ]:
metadata

In [ ]:
target = metadata[["image_id","target"]]

In [ ]:
target

In [ ]:
y = metadata.target

In [ ]:
from keras.utils import to_categorical

y_cat = to_categorical(y)

In [ ]:
y_cat

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import io
import os  # To handle file paths

def resize(width: int, height: int) -> pd.DataFrame:
    pixel_list = []
    image_ids = []

    for idx, image in enumerate(images):
        print(f"Processing image {idx + 1} of {len(images)}: {image}")

        blob = bucket.blob(image)
        image_bytes = blob.download_as_bytes()
        img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        img = img.resize((width, height))
        img_array = np.array(img).flatten()
        pixel_list.append(img_array)

        # Extract image ID (e.g., 'ISIC_0024306')
        image_id = os.path.basename(image).split('.')[0]
        image_ids.append(image_id)

    data = np.stack(pixel_list)
    df = pd.DataFrame(data)
    df.insert(0, "image_id", image_ids)  # Insert image_id as first column

    return df

In [ ]:
df = resize(96,96)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
metadata = get_data_from_bq(f"{GCP_PROJECT}", query_metadata)
metadata.head()

In [ ]:
target = metadata[["image_id","dx","index"]]

In [ ]:
target.dx = target.dx.map(dx_dict)

In [ ]:
target

In [ ]:
df = target.merge(df, how="left", on="image_id")

In [ ]:
df

In [ ]:
df.drop(columns=["image_id","dx"],inplace=True)

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
df = df.astype("int64")

In [ ]:
df.head()

In [ ]:
GCP_PROJECT = os.environ.get("GCP_PROJECT")
GCP_REGION = os.environ.get("GCP_REGION")

In [ ]:
def load_data_to_bq(
        data: pd.DataFrame,
        gcp_project:str,
        bq_dataset:str,
        table: str,
        truncate: bool
    ) -> None:
    """
    - Save the DataFrame to BigQuery
    - Empty the table beforehand if `truncate` is True, append otherwise
    """
    assert isinstance(data, pd.DataFrame)
    full_table_name = f"{gcp_project}.{bq_dataset}.{table}"
    print(Fore.BLUE + f"\nSave data to BigQuery @ {full_table_name}...:" + Style.RESET_ALL)
    client = bigquery.Client(project=gcp_project)
    if truncate == True:
        write_mode = "WRITE_TRUNCATE"
    else:
        write_mode = "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
    job = client.load_table_from_dataframe(data, full_table_name, job_config=job_config)
    result = job.result()

    print(f"✅ Data saved to bigquery, with shape {data.shape}")

In [ ]:
import pandas as pd
from colorama import Fore, Style
from google.cloud import bigquery

In [ ]:
load_data_to_bq(df,f"{GCP_PROJECT}",f"{BQ_DATASET}","train_96_96",True)

In [ ]:
test_array = np.array(df.iloc[0,:])

In [ ]:
test_array = test_array.reshape((96,96,3))

In [ ]:
plt.imshow(test_array)

In [ ]:
plt.imshow(pixel_list[0])

In [ ]:
for image in images:
    blob = bucket.blob(image)
    image_bytes = blob.download_as_bytes()
    img = Image.open(io.BytesIO(image_bytes))
    pixel_list.append(np.array(img))

In [ ]:
pixel_list

In [ ]:
def print_image(name):  
    for image in images:
        if image.endswith(f"{name}.jpg"):
            blob = bucket.blob(image)
            image_bytes = blob.download_as_bytes()
            img = Image.open(io.BytesIO(image_bytes))
            plt.imshow(img)
            plt.axis('off')
            plt.show()
            return
    return "Not found"

In [ ]:
image_resized = image.resize((96,96))

In [ ]:
plt.imshow(image_resized)
plt.show()

In [ ]:
type(image_resized)

In [ ]:
def resize(img: xx, width:int, height:int) -> np.array (width,height,3)

In [ ]:
def get_28_28_image(index):
    temp_array = np.array(data.iloc[index, :]).reshape((28, 28, 3))
    image_id = metadata.iloc[index, :].image_id
    return temp_array, image_id

In [ ]:
def get_cloud_image(name):
    for image in images:
        if image.endswith(f"{name}.jpg"):
            blob = bucket.blob(image)
            image_bytes = blob.download_as_bytes()
            img = Image.open(io.BytesIO(image_bytes))
            return img
    return None

In [ ]:
import random
import matplotlib.pyplot as plt

def show_comparison(n):
    """Shows n random pairs of the original image and the resized 28x28 version"""
    
    # Pick n unique random indices between 0 and 10014 (inclusive)
    random_indices = random.sample(range(0, 10015), n)
    
    fig, axes = plt.subplots(n, 2, figsize=(8, 12))  # n rows, 2 columns
    
    for row_idx, data_idx in enumerate(random_indices):
        local_img, chosen_id = get_28_28_image(data_idx)
        cloud_img = get_cloud_image(chosen_id)
    
        # Left: 28x28 image
        axes[row_idx, 0].imshow(local_img)
        axes[row_idx, 0].set_title(f'Index {data_idx}')
    
        # Right: Cloud image or fallback
        if cloud_img:
            axes[row_idx, 1].imshow(cloud_img)
            axes[row_idx, 1].set_title(chosen_id)
        else:
            axes[row_idx, 1].text(0.5, 0.5, "Not found", ha='center', va='center')
    
    plt.tight_layout()
    plt.show()

In [ ]:
show_comparison(3)